# Контекстно-независимое векторное представление слов 

1. Используйте тексты из предыдущего задания и обучите на них модель векторного представления слов (опционально можно приводить слова к нормальной форме и удалить стоп-слова). Можно использовать `gensim`.

2. Разделите коллекцию текстов на обучающее и тестовое множество. С помощью обученной модели векторного представления отобразите каждый документ в вектор, усреднив все вектора для слов документа. 

3. Используйте какой-либо алгоритм классификации (например `SVM`) для классификации текстов. Для обучения используйте тестовое множество, для анализа результатов - тестовое.

4. Простое усреднение векторов слов обычно не дает хорошего отображения документа в вектор. Придумайте альтернативный способ. Протестируйте его, повторно обучив алгоритм классификации на тех же данных.

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alyonka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#1
  
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import gzip
from nltk.corpus import stopwords


def preprocess(text):
    stop_words = set(stopwords.words('russian'))  
    text_str = ' '.join(text)
    return [word for word in simple_preprocess(text_str) if word not in stop_words]

def load_texts_from_gzip(file_path):
    texts = []
    categories = []
    with gzip.open(file_path, 'rt', encoding='utf-8') as gz_file:
        for line in gz_file:
            category, title, text = line.strip().split('\t')
            full_text = f"{title} {text}"             
            categories.append(category)
            texts.append(simple_preprocess(full_text, deacc=True))  
    return texts, categories

file_path = "D://Nlp'23//nlp-2023//data//news.txt.gz"
texts, categories = load_texts_from_gzip(file_path)


processed_texts = [preprocess(text) for text in texts]


model = Word2Vec(processed_texts, vector_size=100, window=5, min_count=1, workers=4)

model.save('word2vec_model.bin')

vector_science = model.wv['science']
print(vector_science)

[ 0.17487025 -0.21993937  0.23280168 -0.9565174  -0.46915236 -0.5328271
  0.5924817   0.7911074  -0.53437495 -0.14861862  0.37876156 -1.1040527
 -0.6631277   0.22350211  0.2631207  -0.10909087  0.21100278  0.13514163
 -0.05833348 -1.7306471   0.40038756 -0.09758746  0.35301742  0.5998624
 -0.50055236  0.96901643 -0.18253003 -0.8461332  -0.16839561  0.58657
  0.22522123  0.26704293  0.35815004  0.0048836   1.0009377  -0.30500847
  0.5190053  -0.36052087 -1.0223254  -0.46223456  0.50594187 -0.68825585
 -0.4451541   0.2518149   0.9818086  -0.74031323 -0.8163607   0.01008113
 -0.5282565   0.28519088 -0.3386427  -0.3969391  -0.21881814 -0.33663973
 -0.61325014 -0.41186818 -0.3136901  -0.25263292  0.01260973  0.32684645
  0.00834684 -0.31776896  0.11361655  0.3674609  -0.86514896  0.7830459
 -0.15950629  0.7379268   0.2604584   0.30316347  0.10879931  0.6557162
  0.6571732  -0.05581065  0.09422336  0.55482006  0.56319994 -0.05069923
 -0.41331205 -0.17503008 -0.679263    0.3421092   0.1292124

In [3]:
#2

from sklearn.model_selection import train_test_split
import numpy as np


texts_train, texts_test, categories_train, categories_test = train_test_split(processed_texts, categories, test_size=0.2, random_state=42)

def document_vector(model, doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv], axis=0)

train_vectors = [document_vector(model, doc) for doc in texts_train]
test_vectors = [document_vector(model, doc) for doc in texts_test]

print(train_vectors[0])


[-9.8666385e-02  4.9478608e-01  2.6955810e-01  6.8637542e-02
 -1.4511509e-01 -7.1756470e-01  4.6641585e-01  1.0526983e+00
 -3.7028247e-01 -6.2013716e-01 -1.9599545e-01 -9.0326405e-01
 -2.1689889e-01  3.7545747e-01  1.6277935e-01 -4.3027753e-01
  2.5271619e-02 -3.6724231e-01  6.7544994e-03 -1.1860762e+00
  2.2877713e-01 -8.5457589e-04  4.5040783e-01 -3.3930022e-02
 -1.9753447e-01  2.3945320e-01 -3.6652562e-01 -2.5676030e-01
 -3.9200771e-01  3.9809299e-01  4.9960732e-01  2.3476968e-02
  3.6575682e-02 -4.7432020e-01 -6.2942944e-02  3.0887750e-01
  3.0142668e-01 -2.4140991e-01 -5.3324974e-01 -7.4597782e-01
  3.9856958e-01 -5.2596545e-01 -3.3850017e-01  1.5693341e-01
  6.0162336e-01 -3.3280191e-01 -4.6111122e-01  2.2933837e-02
 -1.1193115e-01  3.9802951e-01  3.2288313e-01 -6.0498595e-01
 -3.3605403e-01 -3.3113295e-01 -6.9185495e-01  1.4733934e-01
  2.0853600e-01 -1.7927408e-01 -2.9223657e-01  6.5758258e-02
  1.3364650e-01  1.2357526e-02  3.4264911e-02  1.8912803e-02
 -4.9183595e-01  6.90173

In [4]:
#3 

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_classifier = SVC(kernel='linear')
svm_classifier.fit(train_vectors, categories_train)

predictions = svm_classifier.predict(test_vectors)

accuracy = accuracy_score(categories_test, predictions)
print(f'Accuracy: {accuracy}')

print("Classification Report:")
print(classification_report(categories_test, predictions))

Accuracy: 0.696
Classification Report:
              precision    recall  f1-score   support

    business       0.50      0.01      0.02        79
     culture       0.67      0.67      0.67       279
   economics       0.65      0.88      0.75       266
      forces       0.56      0.65      0.60       149
        life       0.64      0.69      0.67       288
       media       0.71      0.68      0.69       299
     science       0.69      0.69      0.69       288
       sport       0.91      0.92      0.92       276
       style       0.86      0.32      0.46        38
      travel       0.29      0.05      0.09        38

    accuracy                           0.70      2000
   macro avg       0.65      0.56      0.56      2000
weighted avg       0.69      0.70      0.68      2000



Вместо простого усреднения векторов слов, можно использовать взвешенное усреднение, учитывая важность каждого слова. Можно, например, использовать веса, основанные на обратной частоте документов (TF-IDF).

In [5]:
#4
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

train_tfidf = tfidf_vectorizer.fit_transform([' '.join(doc) for doc in texts_train])
test_tfidf = tfidf_vectorizer.transform([' '.join(doc) for doc in texts_test])

svm_classifier_tfidf = SVC(kernel='linear')
svm_classifier_tfidf.fit(train_tfidf, categories_train)

predictions_tfidf = svm_classifier_tfidf.predict(test_tfidf)

accuracy_tfidf = accuracy_score(categories_test, predictions_tfidf)
print(f'Accuracy (TF-IDF): {accuracy_tfidf}')

print("Classification Report:")
print(classification_report(categories_test, predictions_tfidf))


Accuracy (TF-IDF): 0.869
Classification Report:
              precision    recall  f1-score   support

    business       0.90      0.33      0.48        79
     culture       0.90      0.91      0.91       279
   economics       0.78      0.95      0.86       266
      forces       0.82      0.85      0.83       149
        life       0.81      0.90      0.85       288
       media       0.90      0.81      0.85       299
     science       0.87      0.89      0.88       288
       sport       0.97      0.98      0.98       276
       style       0.96      0.71      0.82        38
      travel       0.92      0.61      0.73        38

    accuracy                           0.87      2000
   macro avg       0.88      0.79      0.82      2000
weighted avg       0.87      0.87      0.86      2000



Заметно улучшение в сравнении с предыдущим подходом